In [18]:
cd Proj_341752_337188_250222

/content/drive/MyDrive/Colab Notebooks/EE559-MiniProject/Proj_341752_337188_250222


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
!python test2.py -p Proj_341752_337188_250222\  -d Proj_341752_337188_250222/data

!!! Warning: Project folder name must be in the form Proj_XXXXXX_XXXXXX_XXXXXX

> Testing folder structure ...

> Testing forward dummy input ...

> Testing blocks ...
^C


In [ ]:
cd ..


/content/drive/MyDrive/Colab Notebooks/EE559-MiniProject/Proj_341752_337188_250222


In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[32]:


#from module import Module
import torch
from torch.nn.functional import fold, unfold
import math

epsilon = 1e-6

class Module:

	def forward(self, *input):
		raise NotImplementedError

	def backward(self, *gradwrtoutput):
		raise NotImplementedError

	def param(self):
		return []

class Conv2d(Module):
    def __init__(self, input_shape, out_channels,kernel_size ,bias=True,stride=1, padding=1):
        
        self.in_channels, self.input_height, self.input_width =input_shape
        
        self.stride=stride
        self.padding=padding
        
        self.out_channels=out_channels
        self.kernel_size=kernel_size
        
        self.kernel_shape=(out_channels,self.in_channels,kernel_size[0],kernel_size[1])
        
        output_height=torch.tensor(self.input_height-self.kernel_size[0]+2*self.padding).div(self.stride).add(1).ceil().int()
        output_width=torch.tensor(self.input_width-self.kernel_size[1]+2*self.padding).div(self.stride).add(1).ceil().int()
        
        self.output_shape=(self.out_channels, output_height,output_width )

        self.bias=None
        
        self.x=None
        
        self.w=torch.empty(self.kernel_shape).normal_(0, epsilon)
        
        if bias:
            self.bias=torch.empty(out_channels).normal_(0, epsilon)
            
    def forward(self, x):
                         
        self.x = x
        x_unf = unfold(x , kernel_size = self.kernel_size, padding=self.padding)
        output_unf = self.w.view(self.out_channels,-1)@x_unf+self.bias.view(1 , -1 , 1)
        output= fold(output_unf,(self.output_shape[1],self.output_shape[2]), (1,1) , self.stride )
        
        return output
        
                           
    def backward(self,gradwrtoutput):
        
        self.dl_dw=torch.empty(self.kernel_shape).zero_()
        self.dl_db=torch.empty(self.out_channels).zero_()
        self.dl_dx=torch.empty(input_shape).zero_()

        dl_dz_reshaped = gradwrtoutput.permute(1, 2, 3, 0).reshape(self.out_channels, -1) 
        x_unfolded = unfold(self.x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding) 
        x_unfolded_reshaped =x_unfolded.permute(2, 0, 1).reshape(dl_dz_reshaped.shape[1], -1)


        # Find gradient wrt weights
        self.dl_dw = (dl_dz_reshaped @ x_unfolded_reshaped).reshape(self.w.shape)

        # Find gradient wrt bias
        self.dl_db = gradwrtoutput.sum(axis = (0, 2, 3)) 

        # Find gradient wrt Input
        dl_dx= (self.w.reshape(self.out_channels, -1).t() @ dl_dz_reshaped).reshape(x_unfolded.permute(1, 2, 0).shape).permute(2, 0, 1)

        
        dim_inp = (self.x.shape[2], self.x.shape[3])
        self.dl_dx=fold(dl_dx, dim_inp, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)

        return  self.dl_dx,self.dl_dw,self.dl_db
        
        
        
    
    def param(self):
        return [(self.w, self.dl_dw), (self.bias, self.dl_db)]
        
                           
    
    


# In[33]:


import torch.nn as nn
s_1, s_2 = 100,100
k_1, k_2 = 3,3
bs = 2
ch_in, ch_out = 2, 4
stride = 1

input_shape=(ch_in,s_1,s_2)

# input tensor 
X = torch.empty(bs, ch_in, s_1, s_2).normal_().requires_grad_()
X_copy = X.clone().detach().requires_grad_()

# initialize convolution moduls
conv = Conv2d(input_shape,ch_out, kernel_size = (k_1, k_2), bias=True, stride = stride)

# get weigts and bias
F = conv.w
B = conv.bias
F.requires_grad_()
B.requires_grad_()

# forward
out = conv.forward(X)
out_compare = torch.nn.functional.conv2d(X_copy, F, bias = B, stride = stride, padding=1)

# backward
dL_dX,dL_dF, dL_dB = conv.backward(out/out)
print(dL_dF.shape)
out_compare.backward(out_compare/out_compare)

res1=(out_compare - out).abs().sum()
res1

print('same output of conv: ', (out_compare - out).abs().sum()) 
print('same input gradient: ', (X_copy.grad - dL_dX).abs().sum())
print('same weigth gradient: ',(F.grad-dL_dF).abs().sum() )
print('same bias gradient: ',(B.grad-dL_dB).abs().sum() )


# In[ ]:





# In[ ]:






torch.Size([4, 2, 3, 3])
same output of conv:  tensor(2.4021e-08, grad_fn=<SumBackward0>)
same input gradient:  tensor(4.3979e-08, grad_fn=<SumBackward0>)
same weigth gradient:  tensor(0.0182, grad_fn=<SumBackward0>)
same bias gradient:  tensor(0., grad_fn=<SumBackward0>)
